In [1]:
import pandas as pd

# = = =  = = =  = = =

from selenium.webdriver import ChromeOptions

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

from bs4 import BeautifulSoup
from lxml import etree

# = = =  = = =  = = =

import re

# = = =  = = =  = = =

import json

# = = =  = = =  = = =

import html as html_lib

In [2]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/1.url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：28



,No,Url,Unnamed: 2,Unnamed: 3,Request_Url
0,136,https://www.amazon.com/TWTADE-Control-Momentar...,SWITCH,foot switch,https://www.amazon.com/TWTADE-Control-Momentar...
1,138,https://www.amazon.com/TWTADE-Pneumatic-Positi...,SWITCH,foot switch,https://www.amazon.com/TWTADE-Pneumatic-Positi...
2,280,https://www.amazon.com/3pcs-CBB61-Celiling-Cap...,Capacitance,,https://www.amazon.com/3pcs-CBB61-Celiling-Cap...
3,282,https://www.amazon.com/3pcs-CBB61-Celiling-Cap...,Capacitance,,https://www.amazon.com/3pcs-CBB61-Celiling-Cap...
4,285,https://www.amazon.com/TWTADE-Indicator-%EF%BC...,Signal lamp,,https://www.amazon.com/TWTADE-Indicator-%EF%BC...
5,323,https://www.amazon.com/TWTADE-Conditioner-Swit...,Automobile decoration,Ford F150 6pcs,https://www.amazon.com/TWTADE-Conditioner-Swit...
6,324,https://www.amazon.com/TWTADE-Conditioner-Swit...,Automobile decoration,Ford F150 6pcs,https://www.amazon.com/TWTADE-Conditioner-Swit...
7,325,https://www.amazon.com/TWTADE-Conditioner-Swit...,Automobile decoration,Ford F150 6pcs,https://www.amazon.com/TWTADE-Conditioner-Swit...
8,333,https://www.amazon.com/TWTADE-Conditioner-Cons...,Automobile decoration,Toyota Camry,https://www.amazon.com/TWTADE-Conditioner-Cons...
9,334,https://www.amazon.com/TWTADE-Conditioner-Cons...,Automobile decoration,Toyota Camry,https://www.amazon.com/TWTADE-Conditioner-Cons...


In [3]:
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
# option.add_argument('--headless=new')
# option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument(get_header(method='selenium'))
option.add_argument(get_proxy(method='selenium'))
option.page_load_strategy = 'normal'

driver = Chrome(service=Service('../chromedriver'), options=option)
driver.maximize_window()

# = = =  = = =  = = =

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        driver.get('https://www.amazon.com/?language=en_US')
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

driver

1


<selenium.webdriver.chrome.webdriver.WebDriver (session="e59bd9931d7b76aae1e6522d7436aca0")>

In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                           136
 Url            https://www.amazon.com/TWTADE-Control-Momentar...
 Unnamed: 2                                                SWITCH
 Unnamed: 3                                           foot switch
 Request_Url    https://www.amazon.com/TWTADE-Control-Momentar...
 Name: 0, dtype: object)

In [5]:
url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param['language'] = 'en_US'

url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

crawler_retry = 0
while True:
    crawler_retry += 1

    try:
        driver.get(url_request)
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0.2 * document.body.scrollHeight);")
        time.sleep(1.5)
        driver.execute_script("window.scrollTo(0, 0.45 * document.body.scrollHeight);")
        time.sleep(0.7)
        driver.execute_script("window.scrollTo(0, 0.7 * document.body.scrollHeight);")
        time.sleep(1.2)
        driver.execute_script("window.scrollTo(0, 0.95 * document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")

        soup = BeautifulSoup(driver.page_source, 'lxml')
        html = etree.HTML(str(soup))

        with open('./part.html', 'w', encoding='UTF-8') as file:
            file.write(str(soup))

        break
    except KeyboardInterrupt:
        break
    except:
        continue

url_request

'https://www.amazon.com/TWTADE-Control-Momentary-Industrial-Aluminum/dp/B0885T3QJL?ref_=ast_sto_dp&language=en_US'

In [6]:
zip_code = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', html.xpath('//span[@id="glow-ingress-line2"]/text()')[0]).strip()

zip_code

'Chino 91710'

In [7]:
price = html.xpath('//span[@class="a-offscreen"]/text()')[0].strip() if html.xpath('//span[@class="a-offscreen"]/text()') else ''

price

'$39.95'

In [8]:
dict_src, list_src = {}, json.loads('{"initial":[{' + str(soup).split('\'colorImages\': { \'initial\': [{')[1].split('}]},')[0] + '}]}')['initial']
for i, src in enumerate(list_src):
    dict_src[str(i)] = src['hiRes'].strip() if src['hiRes'] else src['large'].strip()

dict_src

{'0': 'https://m.media-amazon.com/images/I/41FpUnGjsnL.jpg',
 '1': 'https://m.media-amazon.com/images/I/41jK1dCTDEL.jpg',
 '2': 'https://m.media-amazon.com/images/I/41btjVfIhhL.jpg',
 '3': 'https://m.media-amazon.com/images/I/418jOBWTnDL.jpg',
 '4': 'https://m.media-amazon.com/images/I/41tcxM5h24L.jpg'}

In [9]:
dict_key, list_tr = {}, html.xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_overview, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[0]).strip()
    if dict_key[key] == 1:
        dict_overview[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}
    else:
        if key in dict_key:
            dict_key[key] += 1
        else:
            dict_key[key] = 1
        dict_overview[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/span/text()')[1]).strip()}

dict_overview

{'0': {'Operation Mode': 'Foot Switch'},
 '1': {'Current Rating': '15 Amps'},
 '2': {'Operating Voltage': '380 Volts'},
 '3': {'Contact Type': 'Normally Open, Normally Closed'},
 '4': {'Brand': 'TWTADE'},
 '5': {'Terminal': 'Spdt'},
 '6': {'Mounting Type': 'Panel Mount'},
 '7': {'Connectivity Protocol': 'X-10'},
 '8': {'Unit Count': '1 Count'},
 '9': {'Manufacturer': 'TWTADE'}}

In [10]:
about, list_text = '', [re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', text).strip() for text in html.xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]/li/span/text()')]
for i, text in enumerate(list_text):
    about += f'{i + 1}. {text}\n'

print(about.strip())

In [11]:
dict_ = json.loads(re.sub(r'\\(?!["\\])', r'\\\\', str(soup).split('jQuery.parseJSON(\'')[1].split('}\');')[0] + '}'))

dict_

{'dataInJson': None,
 'alwaysIncludeVideo': True,
 'autoplayVideo': False,
 'defaultColor': 'initial',
 'mainImageSizes': [['342', '445'],
  ['385', '500'],
  ['425', '550'],
  ['466', '606'],
  ['522', '679']],
 'maxAlts': 7,
 'altsOnLeft': True,
 'productGroupID': 'biss_basic_display_on_website',
 'lazyLoadExperienceDisabled': True,
 'lazyLoadExperienceOnHoverDisabled': False,
 'useChromelessVideoPlayer': False,
 'colorToAsin': {},
 'refactorEnabled': True,
 'useIV': True,
 'tabletWeb': False,
 'views': ['ImageBlockMagnifierView',
  'ImageBlockAltImageView',
  'ImageBlockVideoView',
  'ImageBlockTwisterView',
  'ImageBlockImmersiveViewImages',
  'ImageBlockImmersiveViewVideos',
  'ImageBlockImmersiveViewDimensionIngress',
  'ImageBlockImmersiveViewShowroom',
  'ImageBlockImmersiveView360',
  'ImageBlockTabbedImmersiveView',
  'ImageBlockShoppableSceneView'],
 'enhancedHoverOverlay': False,
 'landingAsinColor': 'initial',
 'colorImages': {},
 'heroImages': {},
 'enable360Map': {},
 's

In [12]:
title = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', dict_['title']).strip()

title

'TWTADE Heavy Duty Control Nonslip Foot Switch Momentary AC 380V 15A SPDT NO NC Industrial Aluminum Foot Pedal Switch TFS-302'

In [13]:
parent_asin = dict_['parentAsin'].strip()

parent_asin

'B0885T3QJL'

In [14]:
dict_key, list_tr = {}, html.xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_technical, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if dict_key[key] == 1:
        dict_technical[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_technical[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

dict_technical

{}

In [15]:
i, dict_additional, list_tr = 0, {}, html.xpath('//table[@id="productDetails_detailBullets_sections1"]/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip()
    if key == 'Customer Reviews':
        dict_additional[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
        dict_additional[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
        i += 2
    elif key == 'Best Sellers Rank':
        list_span = tr.xpath('./td/span/span')
        list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
        list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
        list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
        list_text = [html_lib.unescape(text) for text in list_text]

        list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
        for j, (text, href) in enumerate(zip(list_text, list_href)):
            if j == 0:
                dict_additional[str(i)] = {'Best Sellers Rank': text}
                dict_additional[str(i + 1)] = {'Best Sellers Rank Url': href}
            else:
                dict_additional[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                dict_additional[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
            i += 2
    else:
        dict_additional[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
        i += 1

dict_additional

{}

In [16]:
dict_key, list_tr = {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/div[@id="detailBullets_feature_div"]/ul/li/span')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_technical_2, dict_count = {}, {}
for i, tr in enumerate(list_tr):
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if dict_key[key] == 1:
        dict_technical_2[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_technical_2[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./span/text()')[1]).strip()}

dict_technical_2

{'0': {'Package Dimensions': '8.43 x 5.39 x 4.37 inches; 1.54 Pounds'},
 '1': {'Date First Available': 'May 6, 2020'},
 '2': {'Manufacturer': 'TWTADE'},
 '3': {'ASIN': 'B0885T3QJL'}}

In [17]:
i, dict_additional_2, list_span = 0, {}, html.xpath('//div[@id="detailBulletsWrapper_feature_div"]/ul[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]/li/span')
for span in list_span:
    if not span.xpath('./span'):
        continue

    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', span.xpath('./span/text()')[0]).strip().removesuffix(':').strip()
    if key == 'Customer Reviews':
        dict_additional_2[str(i)] = {'Customer Reviews Rating Count': span.xpath('./div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
        dict_additional_2[str(i + 1)] = {'Customer Reviews Rating': span.xpath('./div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
        i += 2
    elif key == 'Best Sellers Rank':
        list_span_2 = span.xpath('./descendant-or-self::span[@class="a-list-item"]')
        list_text = [etree.tostring(span_2).decode('utf-8') for span_2 in list_span_2]
        list_text = [re.sub(r'<ul.*?</ul>|<span.*?Best Sellers Rank.*?</span>|<.*?>|\(.*?\)', '', text).strip() for text in list_text]
        list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
        list_text = [html_lib.unescape(text) for text in list_text]

        list_href = [f'https://www.amazon.com{href.strip()}' for href in span.xpath('./descendant-or-self::a/@href')]
        for j, (text, href) in enumerate(zip(list_text, list_href)):
            if j == 0:
                dict_additional_2[str(i)] = {'Best Sellers Rank': text}
                dict_additional_2[str(i + 1)] = {'Best Sellers Rank Url': href}
            else:
                dict_additional_2[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                dict_additional_2[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
            i += 2
    else:
        print('fuck')

dict_additional_2

{'0': {'Customer Reviews Rating Count': '13 ratings'},
 '1': {'Customer Reviews Rating': '4.2 out of 5 stars'}}

In [18]:
dict_key, list_tr = {}, html.xpath('//div[@id="productDetails_expanderSectionTables"]/div[@class="a-row"]/div[contains(@class, "a-column a-span6")]/div[@class="a-row a-spacing-medium-plus"]/div[@class="a-section"]/div[contains(@data-csa-c-content-id, "voyager-expander-btn")]/div/div[@class="a-section"]/table/tbody/tr')
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
    if key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

i, dict_information, dict_count = 0, {}, {}
for tr in list_tr:
    key = re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./th/text()')[0]).strip().removesuffix(':').strip()
    if dict_key[key] == 1:
        if key == 'Customer Reviews':
            dict_information[str(i)] = {'Customer Reviews Rating Count': tr.xpath('./td/div/span[@data-action="acrLink-click-metrics"]/a/span/text()')[0].strip()}
            dict_information[str(i + 1)] = {'Customer Reviews Rating': tr.xpath('./td/div/span[@data-action="acrStarsLink-click-metrics"]/span/@title')[0].strip()}
            i += 2
        elif key == 'Best Sellers Rank':
            list_span = tr.xpath('./td/span/span')
            list_text = [etree.tostring(span).decode('utf-8') for span in list_span]
            list_text = [re.sub(r'<.*?>|\(.*?\)', '', text).strip() for text in list_text]
            list_text = [re.sub(r' +', ' ', text.strip()) for text in list_text]
            list_text = [html_lib.unescape(text) for text in list_text]

            list_href = [f'https://www.amazon.com{href.strip()}' for href in tr.xpath('./td/span/span/a/@href')]
            for j, (text, href) in enumerate(zip(list_text, list_href)):
                if j == 0:
                    dict_information[str(i)] = {'Best Sellers Rank': text}
                    dict_information[str(i + 1)] = {'Best Sellers Rank Url': href}
                else:
                    dict_information[str(i)] = {f'Best Sellers Rank-{j + 1}': text}
                    dict_information[str(i + 1)] = {f'Best Sellers Rank-{j + 1} Url': href}
                i += 2
        else:
            dict_information[str(i)] = {key: re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}
            i += 1
    else:
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1
        dict_information[str(i)] = {f'{key}-{dict_count[key]}': re.sub(r'[\u0020\u00A0\u2000\u2001\u2002\u2003\u2004\u2005\u2006\u2007\u2008\u2009\u200a\u200b\u200c\u200d\u200e\u200f\u202f\u205f\u2060\u2061\u2062\u2063\u2064\u3000\ufeff ]+', ' ', tr.xpath('./td/text()')[0]).strip()}

dict_information

{}

In [19]:
not dict_technical and not dict_additional and not dict_technical_2 and not dict_additional_2 and not dict_information

False

In [20]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Title': title,
                          'About': about.strip(),
                          'Price': price,
                          'Picture': '',
                          'Url': url_request,
                          'Zip Code': zip_code,
                          'Parent ASIN': parent_asin,
                          'Json_Src': json.dumps(dict_src),
                          'Json_Overview': json.dumps(dict_overview),
                          'Json_Technical': json.dumps(dict_technical),
                          'Json_Additional': json.dumps(dict_additional),
                          'Json_Technical 2': json.dumps(dict_technical_2),
                          'Json_Additional 2': json.dumps(dict_additional_2),
                          'Json_Information': json.dumps(dict_information)})

list_dict_correct

[{'No': 136,
  'Title': 'TWTADE Heavy Duty Control Nonslip Foot Switch Momentary AC 380V 15A SPDT NO NC Industrial Aluminum Foot Pedal Switch TFS-302',
  'About': '',
  'Price': '$39.95',
  'Picture': '',
  'Url': 'https://www.amazon.com/TWTADE-Control-Momentary-Industrial-Aluminum/dp/B0885T3QJL?ref_=ast_sto_dp&language=en_US',
  'Zip Code': 'Chino 91710',
  'Parent ASIN': 'B0885T3QJL',
  'Json_Src': '{"0": "https://m.media-amazon.com/images/I/41FpUnGjsnL.jpg", "1": "https://m.media-amazon.com/images/I/41jK1dCTDEL.jpg", "2": "https://m.media-amazon.com/images/I/41btjVfIhhL.jpg", "3": "https://m.media-amazon.com/images/I/418jOBWTnDL.jpg", "4": "https://m.media-amazon.com/images/I/41tcxM5h24L.jpg"}',
  'Json_Overview': '{"0": {"Operation Mode": "Foot Switch"}, "1": {"Current Rating": "15 Amps"}, "2": {"Operating Voltage": "380 Volts"}, "3": {"Contact Type": "Normally Open, Normally Closed"}, "4": {"Brand": "TWTADE"}, "5": {"Terminal": "Spdt"}, "6": {"Mounting Type": "Panel Mount"}, "7": 

In [21]:
crawler_status = True

crawler_status

True

In [22]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                           136
 Url            https://www.amazon.com/TWTADE-Control-Momentar...
 Unnamed: 2                                                SWITCH
 Unnamed: 3                                           foot switch
 Request_Url    https://www.amazon.com/TWTADE-Control-Momentar...
 Name: 0, dtype: object]

In [23]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_part.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Title,About,Price,Picture,Url,Zip Code,Parent ASIN,Json_Src,Json_Overview,Json_Technical,Json_Additional,Json_Technical 2,Json_Additional 2,Json_Information
0,136,TWTADE Heavy Duty Control Nonslip Foot Switch ...,,$39.95,,https://www.amazon.com/TWTADE-Control-Momentar...,Chino 91710,B0885T3QJL,"{""0"": ""https://m.media-amazon.com/images/I/41F...","{""0"": {""Operation Mode"": ""Foot Switch""}, ""1"": ...",{},{},"{""0"": {""Package Dimensions"": ""8.43 x 5.39 x 4....","{""0"": {""Customer Reviews Rating Count"": ""13 ra...",{}


In [24]:
print('Output Finished !')
print()

Output Finished !



In [25]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_part_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Unnamed: 2,Unnamed: 3,Request_Url
0,136,https://www.amazon.com/TWTADE-Control-Momentar...,SWITCH,foot switch,https://www.amazon.com/TWTADE-Control-Momentar...


In [26]:
driver.quit()
print('Done ~')

Done ~
